In [ ]:
from __future__ import print_function
import os,sys
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
from readparquet import UBParquetReader
from detectoroutline import DetectorOutline

In [ ]:
# load utility to draw TPC outline
detdata = DetectorOutline()

# define some colors for particle types
ssnet_color_array = np.array( ((0,0,0),      # bg+other
                               (255,0,0),    # electron
                               (0,255,0),    # gamma
                               (0,0,255),    # muon
                               (255,0,255),  # pion
                               (0,255,255)), # proton
                               dtype=np.float ) 
larcvpid2class = {0:0,#unknown -> bg
                  1:0,#cosmic -> bg
                  2:0,#bnb -> bg
                  3:1,#eminus->electron
                  4:2,#gamma->gamma
                  5:2,#pizero->gamma
                  6:3,#muon->muon
                  7:4,#kaon->other
                  8:4,#pion->pion
                  9:5,#proton->proton
                 }

In [ ]:
# DATA FILES
datafolder="./data/"

reader = UBParquetReader(datafolder)
NENTRIES = reader.nentries

In [ ]:
# Get entry data
ENTRY = 0
data = reader.get_entry(ENTRY)

In [ ]:
# Print Entry Info
for col in ['run','subrun','event','truepos','nu_energy','nu_pid','nu_ccnc','nu_interaction','nu_geniemode']:
    print(col,": ",data[col])


In [ ]:
# print neutrino interaction particle list
for primdata in zip(data['primary_trackid'].as_py(),data['primary_pid'].as_py(),data['primary_start'].as_py(),data['primary_mom'].as_py()):
    trackid=primdata[0]
    print('[%d]'%(trackid),": ",primdata)


In [ ]:
print(data['truetriplet_t'].shape)
print(np.unique( data['segment_t'] ) )
print(np.max(data['kplabel_t']))

In [ ]:
# make spacepoint visualization with particle type labels
no_ghost_points = True
pos = data['spacepoint_t'][:,:3]
label = data['segment_t']
unique_labels = np.unique(label)
color = np.zeros( (pos.shape[0],3), dtype=np.float )
for i in range(10):
   color[ label==i ] = ssnet_color_array[ larcvpid2class[i],:]

if no_ghost_points:
    pos = pos[data['truetriplet_t']==1]
    color = color[data['truetriplet_t']==1]
print(pos.shape)
                 
plot_ssnet = {"type":"scatter3d",
              "x": pos[:,0],
              "y": pos[:,1],
              "z": pos[:,2],
              "mode":"markers",
              "name":"particletype",
              "marker":{"color":color,"size":1,"opacity":0.8}}


# PLOTLY
detlines = detdata.getlines(color=(10,10,10))

# DATA
plotdata = [plot_ssnet] + detdata.getlines(color=(10,10,10))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Particle-type Labels, Spacepoints',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 2, "y": 0.5, "z": 0},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plotdata, layout=layout)
fig.show()

In [ ]:
# make spacepoint visualization with KEYPOINT SCORE LABELS
KPTYPE = 0
KPNAMES = {0:"neutrino",
    1:"track starts",
    2:"track ends",
    3:"shower starts",
    4:"delta starts",
    5:"michel starts"}

no_ghost_points = True
pos = data['spacepoint_t'][:,:3]
label = data['kplabel_t'][:,KPTYPE]
print("max: ",np.max(label))
print("min: ",np.min(label))

if no_ghost_points:
    datapos = pos[data['truetriplet_t']==1]
    color = label[data['truetriplet_t']==1]
else:
    datapos = pos
    color = label
    
print(datapos.shape)
                 
plot_ssnet = {"type":"scatter3d",
              "x": datapos[:,0],
              "y": datapos[:,1],
              "z": datapos[:,2],
              "mode":"markers",
              "name":"particletype",
              "marker":{"color":color,"size":1,"opacity":0.8,'colorscale':'Viridis'}}


# PLOTLY
detlines = detdata.getlines(color=(10,10,10))

# DATA
plotdata = [plot_ssnet] + detdata.getlines(color=(10,10,10))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='%s Keypoint Labels, Spacepoints'%(KPNAMES[KPTYPE]),
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 2, "y": 0.5, "z": 0},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plotdata, layout=layout)
fig.show()

In [ ]:
# Instance ID
# make spacepoint visualization with particle type labels
no_ghost_points = True
pos = data['spacepoint_t'][:,:3]
label = data['instance_t']
unique_labels = np.unique(label)
print("num unique instances: ",unique_labels.shape)
color = np.zeros( (pos.shape[0],3), dtype=np.float )
for n,i in enumerate(unique_labels.tolist()):
   color[ label==i ] = np.random.rand(3)*255

if no_ghost_points:
    datapos = pos[data['truetriplet_t']==1]
    color = color[data['truetriplet_t']==1]
else:
    datapos =  pos
print(datapos.shape)
                 
plot_ssnet = {"type":"scatter3d",
              "x": datapos[:,0],
              "y": datapos[:,1],
              "z": datapos[:,2],
              "mode":"markers",
              "name":"particletype",
              "marker":{"color":color,"size":1,"opacity":0.8}}


# PLOTLY
detlines = detdata.getlines(color=(10,10,10))

# DATA
plotdata = [plot_ssnet] + detdata.getlines(color=(10,10,10))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Instance Labels, Spacepoints',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 2, "y": 0.5, "z": 0},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plotdata, layout=layout)
fig.show()

In [ ]:
# make spacepoint visualization with origin label
no_ghost_points = True
pos = data['spacepoint_t'][:,:3]
label = data['origin_t']
print('unique values: ',np.unique(label))

if no_ghost_points:
    datapos = pos[data['truetriplet_t']==1]
    color = 0.5*label[data['truetriplet_t']==1]
else:
    datapos = pos
    color = 0.5*label
    
print(datapos.shape)
                 
plot_ssnet = {"type":"scatter3d",
              "x": datapos[:,0],
              "y": datapos[:,1],
              "z": datapos[:,2],
              "mode":"markers",
              "name":"particletype",
              "marker":{"color":color,"size":1,"opacity":0.8,'colorscale':'Viridis'}}


# PLOTLY
detlines = detdata.getlines(color=(10,10,10))

# DATA
plotdata = [plot_ssnet] + detdata.getlines(color=(10,10,10))

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='Nu Interaction Origin Label, Spacepoints',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 2, "y": 0.5, "z": 0},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plotdata, layout=layout)
fig.show()

In [ ]:
# U-PLANE
img = data['wireimageplane0'] # stored as sparse matrix
img[:,2] = np.clip(img[:,2],0,100)
print(img.shape)

xaxis = np.linspace( 0, 2400, endpoint=False, num=2400 )
yaxis = np.linspace( 2400, 8448, endpoint=False, num=1008 )
print(yaxis.shape)

denseimg = np.zeros( (1008,2400) )
tmpindex = img.astype(np.int)
denseimg[ tmpindex[:,0], tmpindex[:,1] ] = img[:,2]

layout_yplane = go.Layout(
    title='U-PLANE WIRE IMAGE',
    autosize=True,
    hovermode='closest',
    showlegend=False)
    
heatmap = {
    #"type":"heatmapgl",                                                                                                                                                                  
    "type":"heatmap",
    "z":denseimg,
    "x":xaxis,
    "y":yaxis,
    "colorscale":"Jet",
}

fig = go.Figure(data=[heatmap],layout=layout_yplane)
fig.show()


In [ ]:
# V-PLANE
img = data['wireimageplane1'] # stored as sparse matrix
img[:,2] = np.clip(img[:,2],0,100)
print(img.shape)

xaxis = np.linspace( 0, 2400, endpoint=False, num=2400 )
yaxis = np.linspace( 2400, 8448, endpoint=False, num=1008 )
print(yaxis.shape)

denseimg = np.zeros( (1008,2400) )
tmpindex = img.astype(np.int)
denseimg[ tmpindex[:,0], tmpindex[:,1] ] = img[:,2]

layout_yplane = go.Layout(
    title='V-PLANE WIRE IMAGE',
    autosize=True,
    hovermode='closest',
    showlegend=False)
    
heatmap = {
    #"type":"heatmapgl",                                                                                                                                                                  
    "type":"heatmap",
    "z":denseimg,
    "x":xaxis,
    "y":yaxis,
    "colorscale":"Jet",
}

fig = go.Figure(data=[heatmap],layout=layout_yplane)
fig.show()

In [ ]:
# Y-PLANE
img = data['wireimageplane2'] # stored as sparse matrix
img[:,2] = np.clip(img[:,2],0,100)
print(img.shape)

xaxis = np.linspace( 0, 3456, endpoint=False, num=3456 )
yaxis = np.linspace( 2400, 8448, endpoint=False, num=1008 )
print(yaxis.shape)

denseimg = np.zeros( (1008,3456) )
tmpindex = img.astype(np.int)
denseimg[ tmpindex[:,0], tmpindex[:,1] ] = img[:,2]

layout_yplane = go.Layout(
    title='Y-PLANE WIRE IMAGE',
    autosize=True,
    hovermode='closest',
    showlegend=False)
    
heatmap = {
    #"type":"heatmapgl",                                                                                                                                                                  
    "type":"heatmap",
    "z":denseimg,
    "x":xaxis,
    "y":yaxis,
    "colorscale":"Jet",
}

fig = go.Figure(data=[heatmap],layout=layout_yplane)
fig.show()